In [ ]:
# Instale apenas se necessário
!pip install geemap earthengine-api PyGMTSAR scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.2/104.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 52.7 MB/s eta 0:00:00


In [ ]:
import ee
import geemap

# Caminho do arquivo JSON da service account
SERVICE_ACCOUNT = 'ftp-352@ee-crypto.iam.gserviceaccount.com'
KEY_PATH = '/ee-crypto-e170cb76be79.json'  # Ajuste o caminho conforme necessário

credentials = ee.ServiceAccountCredentials(SERVICE_ACCOUNT, KEY_PATH)
ee.Initialize(credentials)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import geopandas as gpd

# Nome do seu arquivo SHP (ajuste se o nome for diferente)
shapefile_path = "/CE_Municipios_2024.shp"

# Lê todos os municípios do Ceará
gdf = gpd.read_file(shapefile_path)

# Veja as colunas para saber o nome do campo de município
print(gdf.columns)

# Veja exemplos de valores (importante checar se tem acentuação, caixa alta, etc)
print(gdf[['NM_MUN', 'CD_MUN']].head(10))  # Substitua 'NM_MUN' pelo nome do campo correto, se diferente

# Filtre apenas Itapipoca (ajuste o campo conforme sua tabela)
# Certifique-se de que 'NM_MUN' is the correct column name
# Also, consider using .str.upper() to handle case variations
filtro = gdf['NM_MUN'].str.upper() == 'ITAPIPOCA'

# Aplica o filtro booleano para selecionar as linhas
itapipoca = gdf[filtro]

# Confira o resultado
print(itapipoca)

# (Opcional) Salve como novo shapefile ou GeoJSON
itapipoca.to_file("itapipoca.geojson", driver="GeoJSON")

# 3. Carregue como ee.FeatureCollection usando geemap
fc = geemap.geojson_to_ee("itapipoca.geojson")

# 4. Extraia a geometria correta para usar em filterBounds
itapipoca_geom = fc.geometry()


Index(['CD_MUN', 'NM_MUN', 'CD_RGI', 'NM_RGI', 'CD_RGINT', 'NM_RGINT', 'CD_UF',
       'NM_UF', 'SIGLA_UF', 'CD_REGIA', 'NM_REGIA', 'SIGLA_RG', 'CD_CONCU',
       'NM_CONCU', 'AREA_KM2', 'geometry'],
      dtype='object')
        NM_MUN   CD_MUN
0        Jucás  2307403
1   Senador Sá  2312809
2  Morada Nova  2308708
3       Chaval  2303907
4   Pentecoste  2310704
5      Camocim  2302602
6   Chorozinho  2303956
7  Nova Olinda  2309201
8        Choró  2303931
9     Cascavel  2303501
     CD_MUN     NM_MUN  CD_RGI     NM_RGI CD_RGINT   NM_RGINT CD_UF  NM_UF  \
64  2306405  Itapipoca  230002  Itapipoca     2301  Fortaleza    23  Ceará   

   SIGLA_UF CD_REGIA  NM_REGIA SIGLA_RG CD_CONCU   NM_CONCU  AREA_KM2  \
64       CE        2  Nordeste       NE  2306405  Itapipoca  1596.225   

                                             geometry  
64  POLYGON ((-39.5432 -3.08411, -39.54315 -3.0841...  


In [ ]:
print (itapipoca)

     CD_MUN     NM_MUN  CD_RGI     NM_RGI CD_RGINT   NM_RGINT CD_UF  NM_UF  \
64  2306405  Itapipoca  230002  Itapipoca     2301  Fortaleza    23  Ceará   

   SIGLA_UF CD_REGIA  NM_REGIA SIGLA_RG CD_CONCU   NM_CONCU  AREA_KM2  \
64       CE        2  Nordeste       NE  2306405  Itapipoca  1596.225   

                                             geometry  
64  POLYGON ((-39.5432 -3.08411, -39.54315 -3.0841...  


In [ ]:
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(itapipoca_geom) \
    .filterDate('2023-01-01', '2025-01-01') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))

s2_img = s2.median().clip(itapipoca)


In [ ]:
emit = ee.ImageCollection('NASA/EMIT/L2A_RFL') \
    .filterBounds(itapipoca_geom) \
    .filterDate('2023-01-01', '2025-01-01')

emit_img = emit.median().clip(itapipoca)


In [ ]:
s1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(itapipoca_geom) \
    .filterDate('2023-01-01', '2025-01-01') \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))

s1_img = s1.mean().clip(itapipoca)


In [ ]:
emit_bands = ['RFL_B0010', 'RFL_B0050', 'RFL_B0100', 'RFL_B0150', 'RFL_B0200', 'RFL_B0250']
emit_img_sel = emit_img.select(emit_bands)

# Entropia espectral (calculada posteriormente via exportação)


In [ ]:
def add_indices(img):
    ndvi = img.normalizedDifference(['B8', 'B4']).rename('NDVI')
    nbr = img.normalizedDifference(['B8', 'B12']).rename('NBR')
    ndwi = img.normalizedDifference(['B3', 'B8']).rename('NDWI')
    return img.addBands([ndvi, nbr, ndwi])

# Aqui, basta passar a imagem composta (s2_img) diretamente:
s2_indexed = add_indices(s2_img)


In [ ]:
emit_bands = ['RFL_B0010', 'RFL_B0050', 'RFL_B0100', 'RFL_B0150', 'RFL_B0200', 'RFL_B0250']
emit_img_sel = emit_img.select(emit_bands)

# Entropia espectral (calculada posteriormente via exportação)


In [ ]:
def add_indices(img):
    ndvi = img.normalizedDifference(['B8', 'B4']).rename('NDVI')
    nbr = img.normalizedDifference(['B8', 'B12']).rename('NBR')
    ndwi = img.normalizedDifference(['B3', 'B8']).rename('NDWI')
    return img.addBands([ndvi, nbr, ndwi])

# Como s2_img é uma única imagem, aplique a função diretamente
s2_indexed = add_indices(s2_img)

In [ ]:
# Exemplo de exportação para Drive
task = ee.batch.Export.image.toDrive(
    image = emit_img_sel.unmask(),
    description = 'emit_itapipoca',
    scale = 30,
    region = itapipoca_geom, # Use the ee.Geometry object
    fileFormat = 'GeoTIFF'
)
task.start()

EEException: Cannot encode object:      CD_MUN     NM_MUN  CD_RGI     NM_RGI CD_RGINT   NM_RGINT CD_UF  NM_UF  \
64  2306405  Itapipoca  230002  Itapipoca     2301  Fortaleza    23  Ceará   

   SIGLA_UF CD_REGIA  NM_REGIA SIGLA_RG CD_CONCU   NM_CONCU  AREA_KM2  \
64       CE        2  Nordeste       NE  2306405  Itapipoca  1596.225   

                                             geometry  
64  POLYGON ((-39.5432 -3.08411, -39.54315 -3.0841...  